## New Code

#### importing necessary libraries

In [10]:
import socket
import json

import tkinter as tk
from tkinter import *
from tkinter import ttk, messagebox
import qrcode
from PIL import Image, ImageTk
import random
import threading
from speck_algo import encrypt, decrypt
from bank_manager import BankManager

# Load config values
with open("config.json") as f:
    config = json.load(f)

BANK_IP = config["BANK_SERVER_IP"]
BANK_PORT = config["BANK_SERVER_PORT"]
UPI_IP = config["UPI_SERVER_IP"]
UPI_PORT = config["UPI_SERVER_PORT"]

#### Main Page

In [2]:
def show_main_page():
    for widget in root.winfo_children():
        widget.destroy()
    
    root.title("UPI Machine")
    
    frame = tk.Frame(root, padx=20, pady=20)
    frame.pack(expand=True)
    
    label = tk.Label(frame, text="UPI Machine", font=("Arial", 18, "bold"))
    label.grid(row=0, column=0, columnspan=2, pady=10)
    
    register_button = ttk.Button(frame, text="Merchant Registration", command=show_registration_page)
    register_button.grid(row=1, column=0, padx=10, pady=10)
    
    transaction_button = ttk.Button(frame, text="Transaction", command=show_transaction_page)
    transaction_button.grid(row=1, column=1, padx=10, pady=10)


#### Registration page

In [3]:
def show_registration_page():
    ifsc_codes = {
    "HDFC": ["HDFC001", "HDFC002", "HDFC003"],
    "ICICI": ["ICICI001", "ICICI002", "ICICI003"],
    "SBI": ["SBI001", "SBI002", "SBI003"]
    }
    
    def update_ifsc(*args):
        selected_bank = bank_var.get()
        ifsc_dropdown["values"] = ifsc_codes.get(selected_bank, [])
        
    def register_merchant():
        name = name_entry.get()
        password = password_entry.get()
        balance = balance_entry.get()
        ifsc = ifsc_dropdown.get()
        
        if not (name and password and balance and ifsc):
            messagebox.showerror("Error", "All fields are required")
            return
        
        # Validate balance
        try:
            balance = float(balance)
            if balance < 0:
                raise ValueError
        except ValueError:
            messagebox.showerror("Error", "Balance must be a non-negative numeric value")
            return
        
        merchant_details = {
            "type": "merchant",
            "name": name,
            "password": password,
            "balance": balance,
            "ifsc": ifsc
        }
        
        try:
            # Connect to bank server
            client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            client_socket.connect((BANK_IP, BANK_PORT))

            # Send merchant details to bank
            client_socket.send(json.dumps(merchant_details).encode())

            # Receive response (MID)
            response = json.loads(client_socket.recv(1024).decode())
            messagebox.showinfo("Registration Successful", f"Merchant ID: {response['MID']}")
            name_entry.delete(0, tk.END)
            password_entry.delete(0, tk.END)
            balance_entry.delete(0, tk.END)
            bank_var.set('')
            ifsc_dropdown.set('')
            client_socket.close()
        except Exception as e:
            messagebox.showerror("Error", f"Failed to register: {e}")

    for widget in root.winfo_children():
        widget.destroy()
    
    frame = tk.Frame(root, padx=20, pady=20)
    frame.pack(expand=True)
    
    tk.Label(frame, text="Merchant Registration", font=("Arial", 16, "bold")).grid(row=0, columnspan=2, pady=10)
    
    tk.Label(frame, text="Name:").grid(row=1, column=0, pady=5, sticky="e")
    name_entry = ttk.Entry(frame)
    name_entry.grid(row=1, column=1, pady=5)
    
    tk.Label(frame, text="Password:").grid(row=2, column=0, pady=5, sticky="e")
    password_entry = ttk.Entry(frame, show="*")
    password_entry.grid(row=2, column=1, pady=5)
    
    tk.Label(frame, text="Initial Balance:").grid(row=3, column=0, pady=5, sticky="e")
    balance_entry = ttk.Entry(frame)
    balance_entry.grid(row=3, column=1, pady=5)
    
    tk.Label(frame, text="Bank:").grid(row=4, column=0, pady=5, sticky="e")
    bank_var = tk.StringVar()
    bank_dropdown = ttk.Combobox(frame, textvariable=bank_var, values=list(ifsc_codes.keys()))
    bank_dropdown.grid(row=4, column=1, pady=5)
    bank_dropdown.bind("<<ComboboxSelected>>", update_ifsc)
    
    tk.Label(frame, text="IFSC Code:").grid(row=5, column=0, pady=5, sticky="e")
    ifsc_dropdown = ttk.Combobox(frame)
    ifsc_dropdown.grid(row=5, column=1, pady=5)
    
    ttk.Button(frame, text="Register", command=register_merchant).grid(row=6, columnspan=2, pady=10)
    ttk.Button(frame, text="Back", command=show_main_page).grid(row=7, columnspan=2, pady=5)

#### Transaction Page

In [8]:
def show_transaction_page():
    def check_merchant_id(mid):
        try:
            client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            client_socket.connect((BANK_IP, BANK_PORT))  # Connect to Bank Server

            details = {
                "type": "check_merchant",
                "merchant_id": mid
            }

            client_socket.send(json.dumps(details).encode("utf-8"))

            response = client_socket.recv(1024).decode("utf-8")
            client_socket.close()

            return response

        except Exception as e:
            print("Error checking merchant ID:", e)
            return None
        
    def generate_qr():
        merchant_id = merchant_entry.get()
        check_mid = check_merchant_id(merchant_id)
        
        if check_mid != "True":
            messagebox.showerror("Error", "Merchant not registered")
            return
        
        vmid = encrypt(merchant_id)
        
        qr = qrcode.make(vmid)
        qr.save("merchant_qr.png")
        
        img = Image.open("merchant_qr.png")
        img = img.resize((150, 150), Image.LANCZOS)
        img = ImageTk.PhotoImage(img)
        
        qr_label.config(image=img)
        qr_label.image = img

    def validate_trans():
        def server_thread():
            try:
                server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
                server_socket.bind((UPI_IP,UPI_PORT))
                server_socket.listen(5)
                print("UPI Machine ready to receive user transaction...")

                client, addr = server_socket.accept()
                print("Connected with User at", addr)

                data = client.recv(1024).decode()
                trans_details = json.loads(data)
                client.close()
                
                vmid = trans_details["merchant_id"]
                mid = decrypt(vmid)

                # Connect to Bank Server
                try:
                    bank_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
                    bank_socket.connect((BANK_IP, BANK_PORT))

                    bank_socket.send(json.dumps({
                        "type": "transaction",
                        "mmid": trans_details["mmid"],
                        "pin": trans_details["pin"],
                        "amount": trans_details["amount"],
                        "merchant_id": mid[2:]
                    }).encode())

                    response = json.loads(bank_socket.recv(1024).decode())
                    bank_socket.close()

                    # Add to blockchain
                    from bankmanager import BankManager
                    blockchain = BankManager()
                    blockchain.add_transaction(
                        bank_name=response["bank_name"],   # Add this key in bank's response
                        uid=trans_details["mmid"],
                        mid=mid[2:],
                        amount=trans_details["amount"]
                    )

                    root.after(0, lambda: messagebox.showinfo("Transaction Status", response["status"]))

                except Exception as e:
                    err_msg = str(e)
                    root.after(0, lambda: messagebox.showerror("UPI Error", err_msg))

            except Exception as e:
                err_msg = str(e)
                root.after(0, lambda: messagebox.showerror("UPI Error", err_msg))
            
            finally:
                try:
                    server_socket.close()
                    print("server closed")
                except:
                    pass
            
        # Start the server in background
        threading.Thread(target=server_thread, daemon=True).start()

            
    for widget in root.winfo_children():
        widget.destroy()
    
    frame = tk.Frame(root, padx=20, pady=20)
    frame.pack(expand=True)
    
    tk.Label(frame, text="Transaction", font=("Arial", 16, "bold")).grid(row=0, column=0, columnspan=2, pady=10)
    
    tk.Label(frame, text="Merchant ID:").grid(row=1, column=0, pady=5, sticky="e")
    merchant_entry = ttk.Entry(frame)
    merchant_entry.grid(row=1, column=1, pady=5)
    
    ttk.Button(frame, text="Generate QR Code", command=generate_qr).grid(row=2, column=0, padx=10, pady=10)
    
    ttk.Button(frame, text="Ready to Receive", command=validate_trans).grid(row=2, column=1, padx=10, pady=10)

    
    qr_label = tk.Label(frame)
    qr_label.grid(row=3, columnspan=2, pady=10)
    
    ttk.Button(frame, text="Back", command=show_main_page).grid(row=4, columnspan=2, pady=5)

#### Run 

In [9]:
root = tk.Tk()
root.geometry("700x400")
show_main_page()
root.mainloop()

UPI Machine ready to receive user transaction...
Connected with User at ('127.0.0.1', 64451)
server closed
UPI Machine ready to receive user transaction...
Connected with User at ('127.0.0.1', 64464)
server closed
UPI Machine ready to receive user transaction...
Connected with User at ('127.0.0.1', 64467)
server closed
UPI Machine ready to receive user transaction...
Connected with User at ('127.0.0.1', 64474)
server closed
UPI Machine ready to receive user transaction...
Connected with User at ('127.0.0.1', 64476)
server closed
